## 1단계. 환경 설정
필요한 라이브러리를 설치하고, Hugging Face에 로그인하여 Gemma-2 2B 모델에 접근.

(주의: Gemma-2 2B 모델은 사용 전에 Hugging Face에서 사용 약관에 동의해야 한다. 허깅페이스 로그인 후 모델 페이지에서 'Access repository' 버튼을 클릭하여 약관에 동의할 것.)

In [ ]:
# 필수 라이브러리 설치
!pip install transformers langchain chromadb fastapi uvicorn huggingface_hub langchain-community

# Hugging Face에 로그인하여 Gemma 모델 접근 권한 획득
from huggingface_hub import login
login(token="")  # Hugging Face 액세스 토큰 입력

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 10.8 MB/s eta 0:00:0

## 2단계. 모델 및 임베딩 설정
sLM인 Gemma-2 2B 모델과 임베딩 모델인 all-MiniLM-L6-v2을 로드합니다.

In [ ]:
# 필수 라이브러리 임포트
import torch  # torch 임포트 추가
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from huggingface_hub import login

# Hugging Face 토큰 설정
login(token="")  # Hugging Face 토큰 입력

# Gemma-2B 모델 로드
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    torch_dtype=torch.bfloat16
).to("cuda")  # GPU로 로드

# 임베딩 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 3. 데이터 준비 및 벡터 DB 구축
예제 문서를 준비하고, 이를 임베딩하여 ChromaDB에 저장합니다.

In [ ]:
from langchain.vectorstores import Chroma

# 예제 문서
documents = [
    "화재 발생 시 가장 가까운 비상구로 대피하세요.",
    "온도가 비정상적으로 높을 경우 즉시 관리자에게 알리세요.",
    "응급 상황에서는 침착하게 119에 연락하세요."
]

# ChromaDB에 문서 임베딩 및 저장
vectorstore = Chroma.from_texts(documents, embedding=embedding_model, persist_directory="db")
vectorstore.persist()

<ipython-input-4-f94c42b8ee33>:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
!pip install python-multipart

In [ ]:
!pip install pyngrok

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
import logging

# 로깅 설정
logging.basicConfig(level=logging.DEBUG)

# LLM 파이프라인 설정
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    max_length=200,  # 최대 텍스트 길이를 늘림
    max_new_tokens=150  # 새로 생성될 텍스트의 최대 길이 설정
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 벡터 DB 준비 (40개 문장 포함)
documents = [
    "화재 발생 시 가장 가까운 비상구로 대피하세요.",
    "온도가 비정상적으로 높을 경우 즉시 관리자에게 알리세요.",
    "응급 상황에서는 침착하게 119에 연락하세요.",
    "비상 상황에서는 엘리베이터 대신 계단을 이용하세요.",
    "화재가 발생하면 반드시 소화기를 사용하여 초기 진압을 시도하세요.",
    "화재 발생 시 물건을 챙기지 말고 신속히 대피하세요.",
    "비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.",
    "대피 시 문이 뜨거운지 확인하여 화재의 위치를 파악하세요.",
    "응급 상황 시 비상등을 활용해 탈출 경로를 확보하세요.",
    "화재 경보가 울리면 즉시 건물을 떠나야 합니다.",
    "응급 처치가 필요하면 가까운 사람에게 도움을 요청하세요.",
    "화재 발생 시 가능한 한 낮은 자세로 이동하세요.",
    "불길이 강할 경우, 젖은 천으로 코와 입을 가리세요.",
    "화재 경보 시스템이 작동하지 않으면 수동으로 119에 신고하세요.",
    "대피 중 주변 사람들에게 비상구 위치를 알려주세요.",
    "대피 후 반드시 출석체크를 통해 안전 여부를 확인하세요.",
    "비상시 장애인을 우선적으로 대피시켜야 합니다.",
    "대피 시 주변 물건을 치우며 길을 확보하세요.",
    "화재 시 가구는 절대 다시 들어가면 안 됩니다.",
    "대피 시 비상 사다리를 활용하여 안전하게 내려가세요.",
    "화재 초기 진압이 불가능하다면 즉시 대피해야 합니다.",
    "화재 대피 시 창문을 닫아 산소 공급을 차단하세요.",
    "대피 시 주변의 불을 끌 필요는 없으며 신속히 대피하세요.",
    "화재 발생 시 유독가스를 피해 호흡을 최소화하세요.",
    "비상 상황에 대비해 가족과 사전 대피 계획을 수립하세요.",
    "화재 경보 훈련에 참여하여 대피 요령을 익히세요.",
    "화재 시 엘리베이터는 작동 중지될 수 있으니 계단을 이용하세요.",
    "비상 상황 시 사용 가능한 소화기의 위치를 숙지하세요.",
    "가스 누출이 의심되면 즉시 가스를 차단하고 대피하세요.",
    "화재 시 주변 사람과 협력하여 대피 경로를 확보하세요.",
    "건물 내 CCTV는 화재 발생 시 대피 정보를 제공합니다.",
    "대피 중 차분하게 행동하여 혼란을 줄이세요.",
    "화재 초기 진압을 위한 소화기 사용법을 익히세요.",
    "화재 시 대피 경로는 항상 비워 두세요.",
    "화재 대피 시 반려동물을 잊지 말고 함께 대피하세요.",
    "연기가 많은 상황에서는 창문을 열어 환기시키세요.",
    "화재 대피 시 휴대폰 등 개인 물품은 두고 대피하세요.",
    "화재 발생 시 대피 우선순위를 기억하세요.",
    "화재 이후 복귀 전에 반드시 안전 검사를 요청하세요.",
    "화재 경보 오작동 시에도 반드시 대피하는 습관을 가지세요.",
    "비상시에 대피소 위치를 사전에 확인하세요.",
]
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_texts(documents, embedding=embedding_model, persist_directory="db")
vectorstore.persist()

# RetrievalQA 체인 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # 최적 유사도 문서 5개 검색
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# 사용자 입력 및 결과 출력
while True:
    query = input("질문을 입력하세요 ('종료'를 입력하면 종료됩니다): ").strip()
    if query.lower() == "종료":
        print("프로그램을 종료합니다.")
        break

    try:
        # 검색 및 QA 실행
        logging.debug(f"입력된 질문: {query}")
        response = qa_chain({"query": query})
        print(f"\n응답: {response['result']}\n")
    except Exception as e:
        # 오류 발생 시 상세 로그 출력
        logging.error(f"오류 발생: {e}")

질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 화재 대피


Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



응답: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 발생 시 대피 우선순위를 기억하세요.

화재 발생 시 대피 우선순위를 기억하세요.

화재 시 계단 난간을 잡고 이동하세요.

Question: 화재 대피
Helpful Answer: 
* 화재 발생 시 대피 우선순위를 기억하세요.
* 화재 시 계단 난간을 잡고 이동하세요. 




질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 대피


Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



응답: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

화재 대피 시 휴대폰 등 개인 물품은 두고 대피하세요.

화재 대피 시 휴대폰 등 개인 물품은 두고 대피하세요.

화재 대피 시 반려동물을 잊지 말고 함께 대피하세요.

화재 대피 시 반려동물을 잊지 말고 함께 대피하세요.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

Question: 대피
Helpful Answer: 
* 화재 대피 시 휴대폰 등 개인 물품은 두고 대피하세요.
* 화재 대피 시 반려동물을 잊지 말고 함께 대피하세요.
* 화재 시 가구는 절대 다시 들어가면 안 됩니다. 




질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 종료
프로그램을 종료합니다.


In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
import logging

# 로깅 설정
logging.basicConfig(level=logging.DEBUG)

# LLM 파이프라인 설정
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    max_length=200,  # 최대 텍스트 길이를 늘림
    max_new_tokens=150  # 새로 생성될 텍스트의 최대 길이 설정
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 벡터 DB 준비 (40개 문장 포함)
documents = [
    "화재 발생 시 가장 가까운 비상구로 대피하세요.",
    "온도가 비정상적으로 높을 경우 즉시 관리자에게 알리세요.",
    "응급 상황에서는 침착하게 119에 연락하세요.",
    "비상 상황에서는 엘리베이터 대신 계단을 이용하세요.",
    "화재가 발생하면 반드시 소화기를 사용하여 초기 진압을 시도하세요.",
    "화재 발생 시 물건을 챙기지 말고 신속히 대피하세요.",
    "비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.",
    "대피 시 문이 뜨거운지 확인하여 화재의 위치를 파악하세요.",
    "응급 상황 시 비상등을 활용해 탈출 경로를 확보하세요.",
    "화재 경보가 울리면 즉시 건물을 떠나야 합니다.",
    "응급 처치가 필요하면 가까운 사람에게 도움을 요청하세요.",
    "화재 발생 시 가능한 한 낮은 자세로 이동하세요.",
    "불길이 강할 경우, 젖은 천으로 코와 입을 가리세요.",
    "화재 경보 시스템이 작동하지 않으면 수동으로 119에 신고하세요.",
    "대피 중 주변 사람들에게 비상구 위치를 알려주세요.",
    "대피 후 반드시 출석체크를 통해 안전 여부를 확인하세요.",
    "비상시 장애인을 우선적으로 대피시켜야 합니다.",
    "대피 시 주변 물건을 치우며 길을 확보하세요.",
    "화재 시 가구는 절대 다시 들어가면 안 됩니다.",
    "대피 시 비상 사다리를 활용하여 안전하게 내려가세요.",
    "화재 초기 진압이 불가능하다면 즉시 대피해야 합니다.",
    "화재 대피 시 창문을 닫아 산소 공급을 차단하세요.",
    "대피 시 주변의 불을 끌 필요는 없으며 신속히 대피하세요.",
    "화재 발생 시 유독가스를 피해 호흡을 최소화하세요.",
    "비상 상황에 대비해 가족과 사전 대피 계획을 수립하세요.",
    "화재 경보 훈련에 참여하여 대피 요령을 익히세요.",
    "화재 시 엘리베이터는 작동 중지될 수 있으니 계단을 이용하세요.",
    "비상 상황 시 사용 가능한 소화기의 위치를 숙지하세요.",
    "가스 누출이 의심되면 즉시 가스를 차단하고 대피하세요.",
    "화재 시 주변 사람과 협력하여 대피 경로를 확보하세요.",
    "건물 내 CCTV는 화재 발생 시 대피 정보를 제공합니다.",
    "대피 중 차분하게 행동하여 혼란을 줄이세요.",
    "화재 초기 진압을 위한 소화기 사용법을 익히세요.",
    "화재 시 대피 경로는 항상 비워 두세요.",
    "화재 대피 시 반려동물을 잊지 말고 함께 대피하세요.",
    "연기가 많은 상황에서는 창문을 열어 환기시키세요.",
    "화재 대피 시 휴대폰 등 개인 물품은 두고 대피하세요.",
    "화재 발생 시 대피 우선순위를 기억하세요.",
    "화재 이후 복귀 전에 반드시 안전 검사를 요청하세요.",
    "화재 경보 오작동 시에도 반드시 대피하는 습관을 가지세요.",
    "비상시에 대피소 위치를 사전에 확인하세요.",
]
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_texts(documents, embedding=embedding_model, persist_directory="db")
vectorstore.persist()

# RetrievalQA 체인 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # 최적 유사도 문서 5개 검색
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# 사용자 입력 및 결과 출력
while True:
    query = input("질문을 입력하세요 ('종료'를 입력하면 종료됩니다): ").strip()
    if query.lower() == "종료":
        print("프로그램을 종료합니다.")
        break

    try:
        # 검색된 문서와 유사도 점수 가져오기
search_results = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 검색된 문서와 점수 분리
relevant_docs = [result[0] for result in search_results]
scores = [result[1] for result in search_results]

# 검색된 문서와 점수 출력
print("\n[검색된 문서 및 유사도 점수]")
for i, (doc, score) in enumerate(zip(relevant_docs, scores), start=1):
    print(f"{i}. 문서: {doc.page_content}")
    print(f"   스코어: {score:.4f}")  # 소수점 4자리까지 출력

# 문맥 결합
combined_context = "\n".join([doc.page_content for doc in relevant_docs])
print(f"\n[결합된 문맥]\n{combined_context}")

# QA 체인 호출
input_data = {"query": query, "context": combined_context}
response = qa_chain(input_data)

# 응답 출력
print(f"\n응답: {response['result']}\n")


질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 화재 발생


Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



[검색된 문서 및 유사도 점수]
1. 문서: 화재 시 가구는 절대 다시 들어가면 안 됩니다.
   스코어: N/A
2. 문서: 화재 시 계단 난간을 잡고 이동하세요.
   스코어: N/A

[결합된 문맥]
화재 시 가구는 절대 다시 들어가면 안 됩니다.
화재 시 계단 난간을 잡고 이동하세요.

응답: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 시 계단 난간을 잡고 이동하세요.

Question: 화재 발생
Helpful Answer: 
1. 화재 시 가구는 절대 다시 들어가면 안 됩니다.
2. 화재 시 계단 난간을 잡고 이동하세요. 


**Answer:** 
1. 화재 시 가구는 절대 다시 들어가면 안 됩니다.
2. 화재 시 계단 난간을 잡고 이동하세요. 


질문을 입력하세요 ('종료'를 입력하면 종료됩니다): CCTV


Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



[검색된 문서 및 유사도 점수]
1. 문서: 건물 내 CCTV는 화재 발생 시 대피 정보를 제공합니다.
   스코어: N/A
2. 문서: 비상시 장애인을 우선적으로 대피시켜야 합니다.
   스코어: N/A

[결합된 문맥]
건물 내 CCTV는 화재 발생 시 대피 정보를 제공합니다.
비상시 장애인을 우선적으로 대피시켜야 합니다.

응답: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

건물 내 CCTV는 화재 발생 시 대피 정보를 제공합니다.

건물 내 CCTV는 화재 발생 시 대피 정보를 제공합니다.

건물 내 CCTV는 화재 발생 시 대피 정보를 제공합니다.

건물 내 CCTV는 화재 발생 시 대피 정보를 제공합니다.

비상시 장애인을 우선적으로 대피시켜야 합니다.

Question: CCTV
Helpful Answer: 
CCTV는 화재 발생 시 대피 정보를 제공합니다. 

**Explanation:**

The provided text states that CCTV cameras in buildings provide information for evacuation during a fire. 




질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 종료
프로그램을 종료합니다.


In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
import logging

# 로깅 설정
logging.basicConfig(level=logging.DEBUG)

# LLM 파이프라인 설정
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    max_length=200,  # 최대 텍스트 길이를 늘림
    max_new_tokens=150  # 새로 생성될 텍스트의 최대 길이 설정
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 벡터 DB 준비 (40개 문장 포함)
documents = [
    "화재 발생 시 가장 가까운 비상구로 대피하세요.",
    "온도가 비정상적으로 높을 경우 즉시 관리자에게 알리세요.",
    "응급 상황에서는 침착하게 119에 연락하세요.",
    "비상 상황에서는 엘리베이터 대신 계단을 이용하세요.",
    "화재가 발생하면 반드시 소화기를 사용하여 초기 진압을 시도하세요.",
    "화재 발생 시 물건을 챙기지 말고 신속히 대피하세요.",
    "비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.",
    "대피 시 문이 뜨거운지 확인하여 화재의 위치를 파악하세요.",
    "응급 상황 시 비상등을 활용해 탈출 경로를 확보하세요.",
    "화재 경보가 울리면 즉시 건물을 떠나야 합니다.",
    "응급 처치가 필요하면 가까운 사람에게 도움을 요청하세요.",
    "화재 발생 시 가능한 한 낮은 자세로 이동하세요.",
    "불길이 강할 경우, 젖은 천으로 코와 입을 가리세요.",
    "화재 경보 시스템이 작동하지 않으면 수동으로 119에 신고하세요.",
    "대피 중 주변 사람들에게 비상구 위치를 알려주세요.",
    "대피 후 반드시 출석체크를 통해 안전 여부를 확인하세요.",
    "비상시 장애인을 우선적으로 대피시켜야 합니다.",
    "대피 시 주변 물건을 치우며 길을 확보하세요.",
    "화재 시 가구는 절대 다시 들어가면 안 됩니다.",
    "대피 시 비상 사다리를 활용하여 안전하게 내려가세요.",
    "화재 초기 진압이 불가능하다면 즉시 대피해야 합니다.",
    "화재 대피 시 창문을 닫아 산소 공급을 차단하세요.",
    "대피 시 주변의 불을 끌 필요는 없으며 신속히 대피하세요.",
    "화재 발생 시 유독가스를 피해 호흡을 최소화하세요.",
    "비상 상황에 대비해 가족과 사전 대피 계획을 수립하세요.",
    "화재 경보 훈련에 참여하여 대피 요령을 익히세요.",
    "화재 시 엘리베이터는 작동 중지될 수 있으니 계단을 이용하세요.",
    "비상 상황 시 사용 가능한 소화기의 위치를 숙지하세요.",
    "가스 누출이 의심되면 즉시 가스를 차단하고 대피하세요.",
    "화재 시 주변 사람과 협력하여 대피 경로를 확보하세요.",
    "건물 내 CCTV는 화재 발생 시 대피 정보를 제공합니다.",
    "대피 중 차분하게 행동하여 혼란을 줄이세요.",
    "화재 초기 진압을 위한 소화기 사용법을 익히세요.",
    "화재 시 대피 경로는 항상 비워 두세요.",
    "화재 대피 시 반려동물을 잊지 말고 함께 대피하세요.",
    "연기가 많은 상황에서는 창문을 열어 환기시키세요.",
    "화재 대피 시 휴대폰 등 개인 물품은 두고 대피하세요.",
    "화재 발생 시 대피 우선순위를 기억하세요.",
    "화재 이후 복귀 전에 반드시 안전 검사를 요청하세요.",
    "화재 경보 오작동 시에도 반드시 대피하는 습관을 가지세요.",
    "비상시에 대피소 위치를 사전에 확인하세요.",
]
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_texts(documents, embedding=embedding_model, persist_directory="db")
vectorstore.persist()

# RetrievalQA 체인 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # 최적 유사도 문서 5개 검색
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# 사용자 입력 및 결과 출력
while True:
    query = input("질문을 입력하세요 ('종료'를 입력하면 종료됩니다): ").strip()
    if query.lower() == "종료":
        print("프로그램을 종료합니다.")
        break

    try:
        # 검색된 문서와 유사도 점수 가져오기
        search_results = retriever.vectorstore.similarity_search_with_score(query, k=5)

        # 검색된 문서와 점수 분리
        relevant_docs = [result[0] for result in search_results]
        scores = [result[1] for result in search_results]

        # 검색된 문서와 점수 출력
        print("\n[검색된 문서 및 유사도 점수]")
        for i, (doc, score) in enumerate(zip(relevant_docs, scores), start=1):
            print(f"{i}. 문서: {doc.page_content}")
            print(f"   스코어: {score:.4f}")  # 소수점 4자리까지 출력

        # 문맥 결합
        combined_context = "\n".join([doc.page_content for doc in relevant_docs])
        print(f"\n[결합된 문맥]\n{combined_context}")

        # QA 체인 호출
        input_data = {"query": query, "context": combined_context}
        response = qa_chain(input_data)

        # 응답 출력
        print(f"\n응답: {response['result']}\n")
    except Exception as e:
        # 오류 발생 시 상세 로그 출력
        logging.error(f"오류 발생: {e}")

질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 화재 발생


Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



[검색된 문서 및 유사도 점수]
1. 문서: 화재 시 가구는 절대 다시 들어가면 안 됩니다.
   스코어: 0.3303
2. 문서: 화재 시 가구는 절대 다시 들어가면 안 됩니다.
   스코어: 0.3303
3. 문서: 화재 시 가구는 절대 다시 들어가면 안 됩니다.
   스코어: 0.3303
4. 문서: 화재 시 가구는 절대 다시 들어가면 안 됩니다.
   스코어: 0.3303
5. 문서: 화재 시 가구는 절대 다시 들어가면 안 됩니다.
   스코어: 0.3303

[결합된 문맥]
화재 시 가구는 절대 다시 들어가면 안 됩니다.
화재 시 가구는 절대 다시 들어가면 안 됩니다.
화재 시 가구는 절대 다시 들어가면 안 됩니다.
화재 시 가구는 절대 다시 들어가면 안 됩니다.
화재 시 가구는 절대 다시 들어가면 안 됩니다.

응답: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

Question: 화재 발생
Helpful Answer: 화재 발생 시 가구는 절대 다시 들어가면 안 됩니다.




질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 응급 상황


Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



[검색된 문서 및 유사도 점수]
1. 문서: 화재 시 계단 난간을 잡고 이동하세요.
   스코어: 0.4146
2. 문서: 화재 시 계단 난간을 잡고 이동하세요.
   스코어: 0.4146
3. 문서: 화재 초기 진압을 위한 소화기 사용법을 익히세요.
   스코어: 0.4321
4. 문서: 화재 초기 진압을 위한 소화기 사용법을 익히세요.
   스코어: 0.4321
5. 문서: 화재 초기 진압을 위한 소화기 사용법을 익히세요.
   스코어: 0.4321

[결합된 문맥]
화재 시 계단 난간을 잡고 이동하세요.
화재 시 계단 난간을 잡고 이동하세요.
화재 초기 진압을 위한 소화기 사용법을 익히세요.
화재 초기 진압을 위한 소화기 사용법을 익히세요.
화재 초기 진압을 위한 소화기 사용법을 익히세요.

응답: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

화재 시 계단 난간을 잡고 이동하세요.

화재 시 계단 난간을 잡고 이동하세요.

화재 초기 진압을 위한 소화기 사용법을 익히세요.

화재 초기 진압을 위한 소화기 사용법을 익히세요.

화재 초기 진압을 위한 소화기 사용법을 익히세요.

Question: 응급 상황
Helpful Answer: 
* **화재 시 계단 난간을 잡고 이동하세요.**
* **화재 초기 진압을 위한 소화기 사용법을 익히세요.**




질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 종료
프로그램을 종료합니다.


In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.3 MB/s eta 0:00:00


In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_transformers import LongContextReorder
from transformers import pipeline
from rapidfuzz import fuzz
import logging

# 로깅 설정
logging.basicConfig(level=logging.DEBUG)

# 중복 및 유사한 문장 제거 함수
def remove_similar_sentences(sentences, threshold=85):
    unique_sentences = []
    for sentence in sentences:
        if all(fuzz.ratio(sentence, unique) < threshold for unique in unique_sentences):
            unique_sentences.append(sentence)
    return unique_sentences

# 문서 리스트
documents = [
    "화재 발생 시 가장 가까운 비상구로 대피하세요.",
    "온도가 비정상적으로 높을 경우 즉시 관리자에게 알리세요.",
    "응급 상황에서는 침착하게 119에 연락하세요.",
    "비상 상황에서는 엘리베이터 대신 계단을 이용하세요.",
    "화재가 발생하면 반드시 소화기를 사용하여 초기 진압을 시도하세요.",
    "화재 발생 시 물건을 챙기지 말고 신속히 대피하세요.",
    "비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.",
    "대피 시 문이 뜨거운지 확인하여 화재의 위치를 파악하세요.",
    "응급 상황 시 비상등을 활용해 탈출 경로를 확보하세요.",
    "화재 경보가 울리면 즉시 건물을 떠나야 합니다.",
    "지진 발생 시 책상 아래로 들어가 머리를 보호하세요.",
    "가스 누출이 의심되면 창문을 열어 환기를 시키세요.",
    "심폐소생술(CPR)은 응급 상황에서 생명을 구할 수 있는 중요한 기술입니다.",
    "대피 시 주변 물건을 치우며 길을 확보하세요.",
    "화재 시 가구는 절대 다시 들어가면 안 됩니다.",
    "대피 시 비상 사다리를 활용하여 안전하게 내려가세요.",
    "산사태 경고 시 고지대나 안전한 장소로 이동하세요.",
    "응급 상황에서는 의료 도움을 즉시 요청하세요.",
]
# 중복 및 유사한 문장 제거
documents = remove_similar_sentences(documents)

# 임베딩 모델 설정
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")

# Chroma 벡터스토어 생성
vectorstore = Chroma.from_texts(documents, embedding=embedding_model, persist_directory="db")
vectorstore.persist()

# LLM 파이프라인 설정
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    max_length=200,
    max_new_tokens=150
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# RetrievalQA 체인 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# 사용자 입력 및 결과 출력
while True:
    query = input("질문을 입력하세요 ('종료'를 입력하면 종료됩니다): ").strip()
    if query.lower() == "종료":
        print("프로그램을 종료합니다.")
        break

    try:
        # 검색된 문서와 점수 가져오기
        search_results = retriever.vectorstore.similarity_search_with_score(query, k=5)

        # 검색된 문서와 점수 분리
        relevant_docs = [result[0] for result in search_results]
        scores = [result[1] for result in search_results]

        # 검색된 문서와 점수 출력
        print("\n[검색된 문서 및 유사도 점수]")
        for i, (doc, score) in enumerate(zip(relevant_docs, scores), start=1):
            print(f"{i}. 문서: {doc.page_content}")
            print(f"   스코어: {score:.4f}")

        # LongContextReorder를 활용한 문서 재배열
        reordering = LongContextReorder()
        reordered_docs = reordering.transform_documents(relevant_docs)

        # 재배열된 문맥 출력
        combined_context = "\n".join([doc.page_content for doc in reordered_docs])
        print(f"\n[결합된 문맥]\n{combined_context}")

        # QA 체인 호출
        input_data = {"query": query, "context": combined_context}
        response = qa_chain(input_data)

        # 응답 출력
        print(f"\n응답: {response['result']}\n")
    except Exception as e:
        logging.error(f"오류 발생: {e}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 화재 발생 시 엘리베이터를 사용할 수 있나요?


Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



[검색된 문서 및 유사도 점수]
1. 문서: 화재 시 가구는 절대 다시 들어가면 안 됩니다.
   스코어: 7.5604
2. 문서: 화재 발생 시 가장 가까운 비상구로 대피하세요.
   스코어: 8.3743
3. 문서: 비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.
   스코어: 9.2800
4. 문서: 화재 경보가 울리면 즉시 건물을 떠나야 합니다.
   스코어: 10.1654
5. 문서: 가스 누출이 의심되면 창문을 열어 환기를 시키세요.
   스코어: 10.4438

[결합된 문맥]
화재 시 가구는 절대 다시 들어가면 안 됩니다.
비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.
가스 누출이 의심되면 창문을 열어 환기를 시키세요.
화재 경보가 울리면 즉시 건물을 떠나야 합니다.
화재 발생 시 가장 가까운 비상구로 대피하세요.

응답: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 발생 시 가장 가까운 비상구로 대피하세요.

비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.

화재 경보가 울리면 즉시 건물을 떠나야 합니다.

가스 누출이 의심되면 창문을 열어 환기를 시키세요.

Question: 화재 발생 시 엘리베이터를 사용할 수 있나요?
Helpful Answer:  This text does not provide information about the use of elevators during a fire. 


질문을 입력하세요 ('종료'를 입력하면 종료됩니다): 산사태 발생 시


Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



[검색된 문서 및 유사도 점수]
1. 문서: 화재 시 가구는 절대 다시 들어가면 안 됩니다.
   스코어: 15.5374
2. 문서: 화재 발생 시 가장 가까운 비상구로 대피하세요.
   스코어: 15.8427
3. 문서: 비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.
   스코어: 18.2071
4. 문서: 화재 경보가 울리면 즉시 건물을 떠나야 합니다.
   스코어: 18.7012
5. 문서: 대피 시 문이 뜨거운지 확인하여 화재의 위치를 파악하세요.
   스코어: 21.5283

[결합된 문맥]
화재 시 가구는 절대 다시 들어가면 안 됩니다.
비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.
대피 시 문이 뜨거운지 확인하여 화재의 위치를 파악하세요.
화재 경보가 울리면 즉시 건물을 떠나야 합니다.
화재 발생 시 가장 가까운 비상구로 대피하세요.

응답: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

화재 시 가구는 절대 다시 들어가면 안 됩니다.

화재 발생 시 가장 가까운 비상구로 대피하세요.

비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.

화재 경보가 울리면 즉시 건물을 떠나야 합니다.

대피 시 문이 뜨거운지 확인하여 화재의 위치를 파악하세요.

Question: 산사태 발생 시
Helpful Answer: 
*  화재 시 가구는 절대 다시 들어가면 안 됩니다.
*  화재 발생 시 가장 가까운 비상구로 대피하세요.
*  비상구 위치를 사전에 숙지하고 대피 계획을 세우세요.
*  화재 경보가 울리면 즉시 건물을 터나야 합니다.
*  대피 시 문이 뜨거운지 확인하여 화재의 위치를 파악하세요.


**What should you do if a landslide occurs?** 


질문을 